### Configuration et préparation des données

In [ ]:
import dspy
import warnings
warnings.filterwarnings('ignore')

# Configurer le modèle de langage
lm = dspy.LM(
    model='ollama_chat/llama3.1:8b',
    api_base='http://localhost:11434',
    temperature=0.3
)

# Configurer DSPy globalement
dspy.configure(lm=lm)

In [ ]:
# Données d'entraînement
trainset = [
    {"ticket": "Mon ordinateur ne démarre plus depuis ce matin. J'ai une présentation importante dans 2 heures.", "category": "Hardware", "priority": "Urgent"},
    {"ticket": "Je n'arrive pas à me connecter à l'imprimante du 3e étage. Ça peut attendre.", "category": "Peripherals", "priority": "Low"},
    {"ticket": "Le VPN ne fonctionne plus. Impossible d'accéder aux fichiers du serveur.", "category": "Network", "priority": "High"},
    {"ticket": "J'ai oublié mon mot de passe Outlook. Je peux utiliser le webmail.", "category": "Account", "priority": "Medium"},
    {"ticket": "Le site web affiche une erreur 500. Les clients ne peuvent plus commander!", "category": "Application", "priority": "Critical"},
    {"ticket": "Ma souris sans fil ne répond plus bien. Les piles sont faibles.", "category": "Peripherals", "priority": "Low"},
    {"ticket": "Le système de paie ne calcule pas les heures supplémentaires. C'est la fin du mois.", "category": "Application", "priority": "Urgent"},
    {"ticket": "J'aimerais une mise à jour de mon logiciel Adobe quand vous aurez le temps.", "category": "Software", "priority": "Low"},
    {"ticket": "Le serveur de base de données est très lent. Toute la production est impactée.", "category": "Infrastructure", "priority": "Critical"},
    {"ticket": "Je ne reçois plus les emails. J'attends des réponses de fournisseurs.", "category": "Email", "priority": "High"},
    {"ticket": "Mon écran externe ne s'affiche plus. Je peux travailler sur le laptop.", "category": "Hardware", "priority": "Medium"},
    {"ticket": "Le wifi de la salle A ne fonctionne pas. Réunion avec des externes dans 30 min.", "category": "Network", "priority": "Urgent"},
    {"ticket": "Je voudrais installer Slack pour mieux collaborer avec l'équipe.", "category": "Software", "priority": "Medium"},
    {"ticket": "Le système de sauvegarde a échoué cette nuit selon le rapport.", "category": "Infrastructure", "priority": "High"},
    {"ticket": "Mon clavier a une touche qui colle. C'est gérable mais ennuyeux.", "category": "Peripherals", "priority": "Low"}
]

# Données de validation
valset = [
    {"ticket": "Le serveur de fichiers est inaccessible. Personne ne peut travailler.", "category": "Infrastructure", "priority": "Critical"},
    {"ticket": "J'ai besoin d'accès au dossier comptabilité pour l'audit. C'est urgent.", "category": "Account", "priority": "Urgent"},
    {"ticket": "L'écran de mon collègue en vacances clignote. On peut attendre.", "category": "Hardware", "priority": "Low"},
    {"ticket": "Le CRM plante quand j'essaie d'exporter les contacts.", "category": "Application", "priority": "High"},
    {"ticket": "Je voudrais changer ma photo de profil quand vous aurez un moment.", "category": "Account", "priority": "Low"},
    {"ticket": "La vidéoconférence ne fonctionne pas. Réunion avec New York dans 10 minutes!", "category": "Application", "priority": "Critical"},
    {"ticket": "Mon antivirus affiche un message d'expiration mais tout fonctionne.", "category": "Software", "priority": "Medium"}
]

# Catégories et priorités possibles
CATEGORIES = ["Hardware", "Software", "Network", "Application", "Infrastructure", "Account", "Email", "Peripherals"]
PRIORITIES = ["Low", "Medium", "High", "Urgent", "Critical"]

print(f"📊 Données chargées : {len(trainset)} entraînement, {len(valset)} validation")
print(f"📦 {len(CATEGORIES)} catégories, {len(PRIORITIES)} priorités")

# Partie 5 : Les optimiseurs

## 4.1 Introduction: modules vs optimiseurs

Jusqu'à présent, nous avons vu des **modules** (Predict, ChainOfThought, ReAct, etc.). Ces modules **exécutent** des tâches en interrogeant le LLM.

Les **optimiseurs**, quant à eux, **améliorent** les modules en :
- Ajoutant des exemples de démonstration (few-shot learning)
- Optimisant les instructions (prompts)
- Ajustant les paramètres
- Sélectionnant les meilleures configurations

**Analogie** : Si un module est comme un étudiant qui résout un problème, un optimiseur est comme un professeur qui améliore la méthode de l'étudiant en lui montrant des exemples et en affinant ses instructions.

### Principaux optimiseurs DSPy

1. **BootstrapFewShot** - Le plus simple : génère des exemples de démonstration
2. **BootstrapFewShotWithRandomSearch** - Teste plusieurs combinaisons d'exemples
3. **MIPRO** - Optimise à la fois les instructions et les exemples
4. **SignatureOptimizer** - Se concentre uniquement sur l'optimisation des instructions
5. **GEPA** - Le plus sophistiqué : utilise des algorithmes génétiques et la réflexion LLM

Dans cette section, nous allons explorer les optimiseurs 1-4. GEPA sera couvert en détail dans la Partie 7.

## 4.2 BootstrapFewShot: générer des exemples de démonstration

**BootstrapFewShot** est l'optimiseur le plus simple de DSPy. Il fonctionne en :

1. Exécutant votre module sur les données d'entraînement
2. Gardant les prédictions correctes (validées par votre métrique)
3. Utilisant ces prédictions comme exemples de démonstration (few-shot)
4. Injectant ces exemples dans le prompt du module optimisé

**Avantages** :
- Simple à comprendre et à utiliser
- Rapide à exécuter
- Amélioration typique de 5-15%

**Inconvénients** :
- Ne modifie pas les instructions
- Qualité dépend de la qualité des données d'entraînement

### Exemple pratique

In [59]:
from dspy.teleprompt import BootstrapFewShot

print("🔧 Optimisation avec BootstrapFewShot...")

# 1. Préparer les données au format DSPy
train_examples = [
    dspy.Example(
        ticket=ex['ticket'],
        category=ex['category'],
        priority=ex['priority']
    ).with_inputs('ticket')
    for ex in trainset
]

# 2. Créer le module de base
basic_classifier = SimpleTicketClassifier()

# 3. Évaluer AVANT optimisation
print("\n📊 Performance AVANT optimisation:")
score_before = evaluate_module(basic_classifier, val_examples, exact_match_metric)
print(f"   Score: {score_before:.2%}")

# 4. Configurer l'optimiseur BootstrapFewShot
optimizer = BootstrapFewShot(
    metric=exact_match_metric,
    max_bootstrapped_demos=4,  # Nombre d'exemples à générer
    max_labeled_demos=4         # Nombre max d'exemples à utiliser
)

# 5. Compiler (optimiser) le module
print("\n🔄 Compilation avec BootstrapFewShot...")
optimized_classifier = optimizer.compile(
    student=basic_classifier,
    trainset=train_examples
)

# 6. Évaluer APRÈS optimisation
print("\n📊 Performance APRÈS optimisation:")
score_after = evaluate_module(optimized_classifier, val_examples, exact_match_metric)
print(f"   Score: {score_after:.2%}")

# 7. Calculer l'amélioration
improvement = ((score_after - score_before) / score_before) * 100 if score_before > 0 else 0
print(f"\n📈 Amélioration: {improvement:+.1f}%")

🔧 Optimisation avec BootstrapFewShot...


NameError: name 'SimpleTicketClassifier' is not defined

### Inspecter les exemples générés

BootstrapFewShot a ajouté des exemples de démonstration au module. Nous pouvons les visualiser :

In [ ]:
# Inspecter les exemples de démonstration ajoutés par BootstrapFewShot
if hasattr(optimized_classifier, 'classifier'):
    predictor = optimized_classifier.classifier
    
    if hasattr(predictor, 'demos') and predictor.demos:
        print(f"📚 BootstrapFewShot a ajouté {len(predictor.demos)} exemples de démonstration:\n")
        
        for i, demo in enumerate(predictor.demos[:3], 1):  # Afficher les 3 premiers
            print(f"Exemple {i}:")
            print(f"  Ticket: {demo.ticket[:80]}...")
            print(f"  Catégorie: {demo.category}")
            print(f"  Priorité: {demo.priority}")
            print()
    else:
        print("ℹ️ Aucun exemple de démonstration trouvé (ou version DSPy différente)")
else:
    print("ℹ️ Structure du module différente de celle attendue")

## 4.3 MIPRO: optimisation des instructions et exemples

**MIPRO** (Multi-prompt Instruction Proposal Optimizer) est un optimiseur plus avancé qui :

1. **Génère plusieurs variantes d'instructions** pour votre signature
2. **Sélectionne les meilleurs exemples** de démonstration
3. **Teste différentes combinaisons** (instructions × exemples)
4. **Garde la meilleure configuration** selon votre métrique

**Avantages** :
- Optimise à la fois les instructions et les exemples
- Amélioration typique de 10-25%
- Explore l'espace des possibilités de manière systématique

**Inconvénients** :
- Plus lent que BootstrapFewShot (nécessite plus d'appels LLM)
- Peut prendre 10-20 minutes avec Ollama local

### Exemple pratique

In [ ]:
from dspy.teleprompt import MIPRO

print("🚀 Optimisation avec MIPRO...")
print("⏰ Attention : Cela peut prendre 10-20 minutes avec Ollama\n")

# 1. Créer un module de base
mipro_classifier = SimpleTicketClassifier()

# 2. Configurer MIPRO
optimizer = MIPRO(
    metric=exact_match_metric,
    num_candidates=5,           # Nombre de variantes d'instructions à générer
    init_temperature=1.0        # Température pour la génération d'instructions
)

# 3. Compiler avec MIPRO
# Note: MIPRO nécessite à la fois trainset et valset (optionnel)
print("🔄 Compilation avec MIPRO (cela va prendre du temps)...")

try:
    mipro_optimized = optimizer.compile(
        student=mipro_classifier,
        trainset=train_examples,
        max_bootstrapped_demos=3,  # Nombre d'exemples à générer
        max_labeled_demos=3,        # Nombre max d'exemples à utiliser
        requires_permission_to_run=False  # Pas de confirmation interactive
    )
    
    # 4. Évaluer le résultat
    print("\n📊 Performance après MIPRO:")
    score_mipro = evaluate_module(mipro_optimized, val_examples, exact_match_metric)
    print(f"   Score: {score_mipro:.2%}")
    
    improvement_mipro = ((score_mipro - score_before) / score_before) * 100 if score_before > 0 else 0
    print(f"   Amélioration vs baseline: {improvement_mipro:+.1f}%")
    
except Exception as e:
    print(f"⚠️ Erreur lors de l'optimisation MIPRO: {e}")
    print("   MIPRO nécessite une configuration spécifique selon la version de DSPy")
    print("   Consultez la documentation DSPy pour plus de détails.")

## 4.4 Autres optimiseurs importants

### SignatureOptimizer

**SignatureOptimizer** se concentre uniquement sur l'optimisation des instructions de votre signature, sans ajouter d'exemples de démonstration.

**Utilisation** :
```python
from dspy.teleprompt import SignatureOptimizer

optimizer = SignatureOptimizer(
    metric=exact_match_metric,
    breadth=10,  # Nombre de variantes à générer
    depth=3      # Nombre d'itérations de raffinement
)

optimized = optimizer.compile(student=classifier, trainset=train_examples)
```

**Avantages** :
- Rapide (pas de génération d'exemples)
- Améliore la clarté des instructions
- Bon pour les tâches où les exemples ne sont pas critiques

**Inconvénients** :
- Amélioration modeste (5-10%)
- Ne tire pas parti du few-shot learning

### BootstrapFewShotWithRandomSearch

Extension de BootstrapFewShot qui teste plusieurs combinaisons aléatoires d'exemples.

**Utilisation** :
```python
from dspy.teleprompt import BootstrapFewShotWithRandomSearch

optimizer = BootstrapFewShotWithRandomSearch(
    metric=exact_match_metric,
    max_bootstrapped_demos=4,
    num_candidate_programs=8  # Nombre de combinaisons à tester
)

optimized = optimizer.compile(student=classifier, trainset=train_examples)
```

**Avantages** :
- Trouve de meilleures combinaisons d'exemples
- Plus robuste que BootstrapFewShot simple
- Amélioration typique de 8-18%

**Inconvénients** :
- Plus lent que BootstrapFewShot (teste plusieurs combinaisons)

## 4.5 Comparaison des optimiseurs

| Optimiseur | Ce qu'il optimise | Vitesse | Amélioration typique | Complexité | Quand l'utiliser |
|------------|------------------|---------|---------------------|------------|------------------|
| **BootstrapFewShot** | Exemples uniquement | ⚡⚡⚡ Rapide | 5-15% | Simple | Première optimisation, tests rapides |
| **BootstrapFewShotWithRandomSearch** | Exemples (avec recherche) | ⚡⚡ Moyen | 8-18% | Simple | Quand vous avez un peu plus de temps |
| **SignatureOptimizer** | Instructions uniquement | ⚡⚡⚡ Rapide | 5-10% | Moyen | Tâches où les instructions sont critiques |
| **MIPRO** | Instructions + exemples | ⚡ Lent | 10-25% | Avancé | Production, quand vous visez la meilleure performance |
| **GEPA** | Instructions + exemples + réflexion | ⚡ Très lent | 15-30% | Très avancé | Maximum de performance, recherche |

### Guide de sélection

**Pour débuter** :
1. Commencez avec **BootstrapFewShot** pour comprendre le concept
2. Si les résultats sont bons, passez à **BootstrapFewShotWithRandomSearch**
3. Si vous avez du temps, essayez **MIPRO**

**Pour la production** :
- Si vous avez des contraintes de temps : **BootstrapFewShotWithRandomSearch**
- Si vous visez la meilleure performance : **MIPRO** ou **GEPA**
- Si vos instructions sont mal formulées : **SignatureOptimizer** d'abord, puis un autre

**Pour la recherche** :
- **GEPA** pour explorer les limites de performance
- **MIPRO** pour une approche plus systématique

### Stratégie recommandée

1. **Phase 1 - Exploration** : Utilisez BootstrapFewShot pour tester rapidement
2. **Phase 2 - Amélioration** : Passez à MIPRO si les résultats sont prometteurs
3. **Phase 3 - Optimisation finale** : Utilisez GEPA pour maximiser la performance (voir Partie 7)

## 4.6 Introduction à GEPA

**GEPA** (Genetic-Pareto Algorithm) est l'optimiseur le plus sophistiqué de DSPy. Il mérite une section complète (Partie 7) car il combine plusieurs techniques avancées :

### Comment GEPA fonctionne

1. **Algorithmes génétiques** : Évolution de populations de prompts
2. **Réflexion LLM** : Utilise un LLM pour analyser les erreurs et proposer des améliorations
3. **Optimisation Pareto** : Équilibre plusieurs objectifs (précision, concision, etc.)
4. **Itérations adaptatives** : Apprend de ses erreurs pour s'améliorer

### Nouveautés dans DSPy 3.0+

GEPA a été intégré directement dans DSPy et nécessite maintenant :
- Un **reflection_lm** : modèle dédié à l'analyse des erreurs
- Configuration via **auto** : 'light', 'medium', ou 'heavy'
- Métrique compatible avec les nouveaux paramètres

### Aperçu rapide

```python
from dspy.teleprompt import GEPA

# Configuration du modèle de réflexion
reflection_lm = dspy.LM(
    model='ollama_chat/llama3.1:8b',
    api_base='http://localhost:11434',
    temperature=1.0,
    max_tokens=8000
)

# Optimiseur GEPA
optimizer = GEPA(
    metric=exact_match_metric,
    auto='light',  # 'light', 'medium', ou 'heavy'
    reflection_lm=reflection_lm
)

# Compilation
optimized = optimizer.compile(
    student=classifier,
    trainset=train_examples,
    valset=val_examples
)
```

### Quand utiliser GEPA

- ✅ Vous avez du temps (15-30 minutes minimum)
- ✅ Vous visez la meilleure performance possible
- ✅ Vous avez suffisamment de données d'entraînement (20+ exemples)
- ✅ Votre métrique est bien définie
- ✅ Vous êtes prêt à expérimenter avec les paramètres

**La Partie 7 de ce tutoriel couvre GEPA en profondeur avec des exemples pratiques, des comparaisons de paramètres, et des conseils d'optimisation.**

## 4.7 Résumé de la Partie 4

### Ce que nous avons appris

1. **Différence modules vs optimiseurs** :
   - Les modules **exécutent** des tâches
   - Les optimiseurs **améliorent** les modules

2. **Les optimiseurs principaux** :
   - **BootstrapFewShot** : Simple, rapide, génère des exemples
   - **BootstrapFewShotWithRandomSearch** : Teste plusieurs combinaisons
   - **SignatureOptimizer** : Optimise uniquement les instructions
   - **MIPRO** : Optimise instructions + exemples
   - **GEPA** : Le plus sophistiqué (détails en Partie 7)

3. **Comment choisir** :
   - Débuter : BootstrapFewShot
   - Production rapide : BootstrapFewShotWithRandomSearch
   - Meilleure performance : MIPRO ou GEPA
   - Instructions mal formulées : SignatureOptimizer

### Points clés à retenir

- ✅ Toujours évaluer **avant** et **après** optimisation
- ✅ Commencer simple, puis complexifier
- ✅ Les optimiseurs nécessitent des données d'entraînement de qualité
- ✅ Plus de temps d'optimisation = généralement meilleure performance
- ✅ Votre métrique détermine ce que l'optimiseur optimise

### Prochaines étapes

- **Partie 5** : Multi-modèles (utiliser différents LLMs)
- **Partie 6** : Patterns avancés (optionnel)
- **Partie 7** : GEPA en pratique (optimisation approfondie)